In [1]:
# Install required libraries
!pip install requests beautifulsoup4 pandas

DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/beautifulsoup4-4.10.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [3]:
url = "https://www.investorgain.com/report/ipo-performance-history/486/ipo/?year=2024"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")


In [4]:
# Find the main table
table = soup.find("table", {"id": "mainTable"})
rows = table.find_all("tr")[1:]  # Skip the header row

data = []

# Extract rows data
for row in rows:
    cells = row.find_all("td")
    ipo_data = {
        "IPO": cells[0].get_text(strip=True),
        "Listing Date": cells[1].get_text(strip=True),
        "IPO_Size": cells[2].get_text(strip=True),
        "Subscription": cells[3].get_text(strip=True),
        "GMP": cells[4].get_text(strip=True),
        "IPO Price": cells[5].get_text(strip=True),
        "Listing Price": cells[6].get_text(strip=True),
        "Closing Price": cells[7].get_text(strip=True),
        "LTP": cells[8].get_text(strip=True)
    }
    data.append(ipo_data)


In [ ]:
# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv("ipo_performance.csv", index=False)

# Display DataFrame
import ace_tools as tools; tools.display_dataframe_to_user("IPO Performance Data", df)


ModuleNotFoundError: No module named 'ace_tools'

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL to scrape
url = "https://www.investorgain.com/report/ipo-gmp-performance-tracker/377/all/?year=2020"

# Send a request to the website
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Find the table by its ID
table = soup.find("table", id="mainTable")

# Extract headers
headers = [header.text.strip() for header in table.find_all("th")]

# Extract rows
data = []
for row in table.find("tbody").find_all("tr"):
    cols = [col.text.strip() for col in row.find_all("td")]
    data.append(cols)

# Create DataFrame
df = pd.DataFrame(data, columns=headers)

# Save to CSV
csv_file = "ipo_performance_2020.csv"
df.to_csv(csv_file, index=False)

csv_file


'ipo_performance_2020.csv'

In [12]:
import pandas as pd

# Load the CSV file
input_file = "/Users/hemantg/Desktop/Finance Sem5/ipo_performance_2024.csv"  # Replace with your input file name
output_file = "processed_ipo_data_2024.csv"  # Replace with your desired output file name

# Read the CSV file
df = pd.read_csv(input_file)

# Remove '‚Çπ ' from all text in the dataframe
df = df.replace({'‚Çπ ': ''}, regex=True)

# Extract details from the 'IPO' column
df[['Name', 'Tickers']] = df['IPO'].str.extract(r'^(.*?) IPO \((.*?)\)$')
df['Tickers'] = df['Tickers'].fillna('')  # Handle empty brackets
df['NSE Ticker'] = 'N/A'
df['BSE Ticker'] = 'N/A'

# Handle cases with and without commas in the 'Tickers' column
for idx, row in df.iterrows():
    tickers = row['Tickers']
    if ',' in tickers:  # Case: Both NSE and BSE tickers exist
        nse, bse = tickers.split(',', 1)
        df.at[idx, 'NSE Ticker'] = nse.strip()
        df.at[idx, 'BSE Ticker'] = bse.strip()
    elif tickers:  # Case: Only one value in the bracket
        if tickers[0].isalpha():  # Starts with a letter
            df.at[idx, 'NSE Ticker'] = tickers.strip()
        elif tickers[0].isdigit():  # Starts with a digit
            df.at[idx, 'BSE Ticker'] = tickers.strip()

# Drop unnecessary columns
df.drop(columns=['IPO', 'Tickers'], inplace=True)

# Extract values and percentages for Listing Price and Closing Price
df[['Listing Price Value', 'Listing Price %']] = df['Listing Price'].str.extract(r'^([\d.]+)\s*\(([^)]+)\)$')
df[['Closing Price Value', 'Closing Price %']] = df['Closing Price'].str.extract(r'^([\d.]+)\s*\(([^)]+)\)$')
df.drop(columns=['Listing Price', 'Closing Price', 'LTP'], inplace=True)

# Rename columns for clarity
df.rename(columns={
    'Listing Date': 'Listing Date',
    'IPO_Size': 'IPO Size',
    'Subscription': 'Subscription',
    'GMP': 'GMP',
    'IPO Price': 'IPO Price',
    'Estimated Price': 'Estimated Price'
}, inplace=True)

# Save the transformed data to a new CSV file
df.to_csv(output_file, index=False)

print(f"Processed data saved to {output_file}")


Processed data saved to processed_ipo_data_2024.csv


In [16]:
import pandas as pd

# Load the CSV file
input_file = "/Users/hemantg/Desktop/Finance Sem5/combined_ipo_performance_final.csv"  # Replace with your input file name
output_file = "processed_ipo_data_all.csv"  # Replace with your desired output file name

# Read the CSV file
df = pd.read_csv(input_file)

# Remove '‚Çπ ' from all text in the dataframe
df = df.replace({'‚Çπ ': ''}, regex=True)

# Extract details from the 'IPO' column
df[['Name', 'Tickers']] = df['IPO'].str.extract(r'^(.*?) IPO \((.*?)\)$')
df['Tickers'] = df['Tickers'].fillna('')  # Handle empty brackets
df['NSE Ticker'] = 'N/A'
df['BSE Ticker'] = 'N/A'

# Handle cases with and without commas in the 'Tickers' column
for idx, row in df.iterrows():
    tickers = row['Tickers']
    if ',' in tickers:  # Case: Both NSE and BSE tickers exist
        nse, bse = tickers.split(',', 1)
        df.at[idx, 'NSE Ticker'] = nse.strip()
        df.at[idx, 'BSE Ticker'] = bse.strip()
    elif tickers:  # Case: Only one value in the bracket
        if tickers[0].isalpha():  # Starts with a letter
            df.at[idx, 'NSE Ticker'] = tickers.strip()
        elif tickers[0].isdigit():  # Starts with a digit
            df.at[idx, 'BSE Ticker'] = tickers.strip()

# Drop unnecessary columns
df.drop(columns=['IPO', 'Tickers'], inplace=True)

# Extract values and percentages for Listing Price, Closing Price, and Estimated Price
df[['Listing Price Value', 'Listing Price %']] = df['Listing Price'].str.extract(r'^([\d.]+)\s*\(([^)]+)\)$')
df[['Closing Price Value', 'Closing Price %']] = df['Closing Price'].str.extract(r'^([\d.]+)\s*\(([^)]+)\)$')
df[['Estimated Price Value', 'Estimated Price %']] = df['Estimated Price'].str.extract(r'^([\d.]+)\s*\(([^)]+)\)$')

# If no brackets exist, keep the original value in 'Estimated Price Value' and set 'Estimated Price %' to NaN
df.loc[df['Estimated Price Value'].isnull(), 'Estimated Price Value'] = df['Estimated Price']
df['Estimated Price %'] = df['Estimated Price %'].fillna('N/A')

# Drop original columns that are split
df.drop(columns=['Listing Price', 'Closing Price', 'Estimated Price', 'LTP'], inplace=True)

# Rename columns for clarity
df.rename(columns={
    'Listing Date': 'Listing Date',
    'IPO_Size': 'IPO Size',
    'Subscription': 'Subscription',
    'GMP': 'GMP',
    'IPO Price': 'IPO Price'
}, inplace=True)

# Save the transformed data to a new CSV file
df.to_csv(output_file, index=False)

print(f"Processed data saved to {output_file}")


Processed data saved to processed_ipo_data_all.csv


In [17]:
import pandas as pd

# Load the CSV file
input_file = "/Users/hemantg/Desktop/Finance Sem5/combined_ipo_performance_final.csv"  # Replace with your input file name
output_file = "processed_ipo_data_all_final.csv"  # Replace with your desired output file name

# Read the CSV file
df = pd.read_csv(input_file)

# Remove '‚Çπ ' from all text in the dataframe
df = df.replace({'‚Çπ ': ''}, regex=True)

# Extract details from the 'IPO' column
df[['Name', 'Tickers']] = df['IPO'].str.extract(r'^(.*?) IPO \((.*?)\)$')
df['Tickers'] = df['Tickers'].fillna('')  # Handle empty brackets
df['NSE Ticker'] = 'N/A'
df['BSE Ticker'] = 'N/A'

# Handle cases with and without commas in the 'Tickers' column
for idx, row in df.iterrows():
    tickers = row['Tickers']
    if ',' in tickers:  # Case: Both NSE and BSE tickers exist
        nse, bse = tickers.split(',', 1)
        df.at[idx, 'NSE Ticker'] = nse.strip()
        df.at[idx, 'BSE Ticker'] = bse.strip()
    elif tickers:  # Case: Only one value in the bracket
        if tickers[0].isalpha():  # Starts with a letter
            df.at[idx, 'NSE Ticker'] = tickers.strip()
        elif tickers[0].isdigit():  # Starts with a digit
            df.at[idx, 'BSE Ticker'] = tickers.strip()

# Drop unnecessary columns
df.drop(columns=['IPO', 'Tickers'], inplace=True)

# Extract values and percentages for Listing Price and Closing Price
df[['Listing Price Value', 'Listing Price %']] = df['Listing Price'].str.extract(r'^([\d.]+)\s*\(([^)]+)\)$')
df[['Closing Price Value', 'Closing Price %']] = df['Closing Price'].str.extract(r'^([\d.]+)\s*\(([^)]+)\)$')
df.drop(columns=['Listing Price', 'Closing Price', 'LTP'], inplace=True)

# Rename columns for clarity
df.rename(columns={
    'Listing Date': 'Listing Date',
    'IPO_Size': 'IPO Size',
    'Subscription': 'Subscription',
    'GMP': 'GMP',
    'IPO Price': 'IPO Price',
    'Estimated Price': 'Estimated Price'
}, inplace=True)

# Save the transformed data to a new CSV file
df.to_csv(output_file, index=False)

print(f"Processed data saved to {output_file}")


Processed data saved to processed_ipo_data_all_final.csv


In [21]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime

# Load CSV
csv_file = '/Users/hemantg/Desktop/Finance Sem5/screener-scraping-try.csv'  # Replace with your CSV file name
data = pd.read_csv(csv_file)

# Define function to scrape tables from Screener

def scrape_tables(nse_ticker, listing_date):
    url = f"https://www.screener.in/company/{nse_ticker}/consolidated/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Convert listing date to datetime
    listing_date = datetime.strptime(listing_date, "%d-%b-%y")

    # Find all tables on the page
    sections = soup.find_all('section', class_='card card-large')

    for section in sections:
        table_title = section.find('h2').text.strip().replace(' ', '_').lower()
        table = section.find('table', class_='data-table')

        # Extract headers and rows
        headers = [th.text.strip() for th in table.find('thead').find_all('th')]
        rows = table.find('tbody').find_all('tr')

        data = []
        for row in rows:
            cols = [td.text.strip() for td in row.find_all('td')]
            data.append(cols)

        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)
        
        # Filter columns based on Listing Date
        columns_to_keep = [headers[0]]
        for col in headers[1:]:
            try:
                col_date = datetime.strptime(col, "%b %Y")
                if col_date <= listing_date:
                    columns_to_keep.append(col)
            except ValueError:
                columns_to_keep.append(col)  # Keep non-date columns

        df = df[columns_to_keep]

        # Update column names and row names
        df.columns = [f'{row}_{col}' if col != headers[0] else col for col in df.columns]
        df.insert(0, 'NSE Ticker', nse_ticker)
        df = df.set_index('NSE Ticker')

        # Save DataFrame to CSV
        output_file = f"{nse_ticker}_{table_title}.csv"
        df.to_csv(output_file, index=True)
        print(f"Saved {output_file}")

# Process each company in the CSV
for index, row in data.iterrows():
    nse_ticker = row['NSE Ticker']
    listing_date = row['Listing Date']
    try:
        scrape_tables(nse_ticker, listing_date)
    except Exception as e:
        print(f"Failed to scrape {nse_ticker}: {e}")


Failed to scrape SWIGGY: 'NoneType' object has no attribute 'text'
Failed to scrape ROSSARI: 'NoneType' object has no attribute 'text'


In [22]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime

# Load CSV
csv_file = '/Users/hemantg/Desktop/Finance Sem5/screener-scraping-try.csv'  # Replace with your CSV file name
data = pd.read_csv(csv_file)

# Define function to scrape tables from Screener

def scrape_tables(nse_ticker, listing_date):
    url = f"https://www.screener.in/company/{nse_ticker}/consolidated/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Convert listing date to datetime
    listing_date = datetime.strptime(listing_date, "%d-%b-%y")

    # Find all tables on the page
    sections = soup.find_all('section', class_='card card-large')

    for section in sections:
        table_title_tag = section.find('h2')
        if not table_title_tag:
            print(f"Table title not found for {nse_ticker}")
            continue

        table_title = table_title_tag.text.strip().replace(' ', '_').lower()
        table = section.find('table', class_='data-table')
        if not table:
            print(f"Table not found for {nse_ticker} in section {table_title}")
            continue

        # Extract headers and rows
        headers = [th.text.strip() for th in table.find('thead').find_all('th')]
        rows = table.find('tbody').find_all('tr')

        data = []
        for row in rows:
            cols = [td.text.strip() for td in row.find_all('td')]
            data.append(cols)

        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)
        
        # Filter columns based on Listing Date
        columns_to_keep = [headers[0]]
        for col in headers[1:]:
            try:
                col_date = datetime.strptime(col, "%b %Y")
                if col_date <= listing_date:
                    columns_to_keep.append(col)
            except ValueError:
                columns_to_keep.append(col)  # Keep non-date columns

        df = df[columns_to_keep]

        # Update column names and row names
        df.columns = [f'{row}_{col}' if col != headers[0] else col for col in df.columns]
        df.insert(0, 'NSE Ticker', nse_ticker)
        df = df.set_index('NSE Ticker')

        # Save DataFrame to CSV
        output_file = f"{nse_ticker}_{table_title}.csv"
        df.to_csv(output_file, index=True)
        print(f"Saved {output_file}")

# Process each company in the CSV
for index, row in data.iterrows():
    nse_ticker = row['NSE Ticker']
    listing_date = row['Listing Date']
    try:
        scrape_tables(nse_ticker, listing_date)
    except Exception as e:
        print(f"Failed to scrape {nse_ticker}: {e}")


Table title not found for SWIGGY
Table title not found for SWIGGY
Table not found for SWIGGY in section peer_comparison
Saved SWIGGY_quarterly_results.csv
Saved SWIGGY_profit_&_loss.csv
Saved SWIGGY_balance_sheet.csv
Saved SWIGGY_cash_flows.csv
Saved SWIGGY_ratios.csv
Saved SWIGGY_shareholding_pattern.csv
Table not found for SWIGGY in section documents
Table title not found for ROSSARI
Table title not found for ROSSARI
Table not found for ROSSARI in section peer_comparison
Saved ROSSARI_quarterly_results.csv
Saved ROSSARI_profit_&_loss.csv
Saved ROSSARI_balance_sheet.csv
Saved ROSSARI_cash_flows.csv
Saved ROSSARI_ratios.csv
Saved ROSSARI_shareholding_pattern.csv
Table not found for ROSSARI in section documents


In [23]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime

# Load CSV
csv_file = '/Users/hemantg/Desktop/Finance Sem5/screener-scraping-try.csv' # Replace with your CSV file name
data = pd.read_csv(csv_file)

# Define function to scrape tables from Screener

def scrape_tables(nse_ticker, listing_date):
    url = f"https://www.screener.in/company/{nse_ticker}/consolidated/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Convert listing date to datetime
    listing_date = datetime.strptime(listing_date, "%d-%b-%y")

    # Create folder for the ticker
    folder_name = nse_ticker
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Find all tables on the page
    sections = soup.find_all('section', class_='card card-large')

    for section in sections:
        table_title_tag = section.find('h2')
        if not table_title_tag:
            print(f"Table title not found for {nse_ticker}")
            continue

        table_title = table_title_tag.text.strip().replace(' ', '_').lower()
        table = section.find('table', class_='data-table')
        if not table:
            print(f"Table not found for {nse_ticker} in section {table_title}")
            continue

        # Extract headers and rows
        headers = [th.text.strip() for th in table.find('thead').find_all('th')]
        rows = table.find('tbody').find_all('tr')

        data = []
        for row in rows:
            cols = [td.text.strip() for td in row.find_all('td')]
            data.append(cols)

        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)
        
        # Filter columns based on Listing Date
        columns_to_keep = [headers[0]]
        for col in headers[1:]:
            try:
                col_date = datetime.strptime(col, "%b %Y")
                if col_date <= listing_date:
                    columns_to_keep.append(col)
            except ValueError:
                columns_to_keep.append(col)  # Keep non-date columns

        df = df[columns_to_keep]

        # Save DataFrame to CSV
        output_file = os.path.join(folder_name, f"{table_title}.csv")
        df.to_csv(output_file, index=False)
        print(f"Saved {output_file}")

# Process each company in the CSV
for index, row in data.iterrows():
    nse_ticker = row['NSE Ticker']
    listing_date = row['Listing Date']
    try:
        scrape_tables(nse_ticker, listing_date)
    except Exception as e:
        print(f"Failed to scrape {nse_ticker}: {e}")


Table title not found for SWIGGY
Table title not found for SWIGGY
Table not found for SWIGGY in section peer_comparison
Saved SWIGGY/quarterly_results.csv
Saved SWIGGY/profit_&_loss.csv
Saved SWIGGY/balance_sheet.csv
Saved SWIGGY/cash_flows.csv
Saved SWIGGY/ratios.csv
Saved SWIGGY/shareholding_pattern.csv
Table not found for SWIGGY in section documents
Table title not found for ROSSARI
Table title not found for ROSSARI
Table not found for ROSSARI in section peer_comparison
Saved ROSSARI/quarterly_results.csv
Saved ROSSARI/profit_&_loss.csv
Saved ROSSARI/balance_sheet.csv
Saved ROSSARI/cash_flows.csv
Saved ROSSARI/ratios.csv
Saved ROSSARI/shareholding_pattern.csv
Table not found for ROSSARI in section documents


In [24]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime

# Load CSV
csv_file = '/Users/hemantg/Desktop/Finance Sem5/mainboard_ipo_final.csv'  # Replace with your CSV file name
data = pd.read_csv(csv_file)

# Define function to scrape tables from Screener

def scrape_tables(nse_ticker, listing_date):
    url = f"https://www.screener.in/company/{nse_ticker}/consolidated/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Convert listing date to datetime
    listing_date = datetime.strptime(listing_date, "%d-%b-%y")

    # Create folder for the ticker
    folder_name = nse_ticker
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Find all tables on the page
    sections = soup.find_all('section', class_='card card-large')

    for section in sections:
        table_title_tag = section.find('h2')
        if not table_title_tag:
            print(f"Table title not found for {nse_ticker}")
            continue

        table_title = table_title_tag.text.strip().replace(' ', '_').lower()
        if 'shareholding_pattern' in table_title:
            continue

        table = section.find('table', class_='data-table')
        if not table:
            print(f"Table not found for {nse_ticker} in section {table_title}")
            continue

        # Extract headers and rows
        headers = [th.text.strip().replace('+', '') for th in table.find('thead').find_all('th')]
        rows = table.find('tbody').find_all('tr')

        data = []
        for row in rows:
            cols = [td.text.strip() for td in row.find_all('td')]
            data.append(cols)

        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)

        # Remove 'Raw PDF' row in Quarterly Results
        if 'quarterly_results' in table_title:
            df = df[df[headers[0]] != 'Raw PDF']
        
        # Filter columns based on Listing Date
        columns_to_keep = [headers[0]]
        for col in headers[1:]:
            try:
                col_date = datetime.strptime(col, "%b %Y")
                if col_date <= listing_date:
                    columns_to_keep.append(col)
            except ValueError:
                columns_to_keep.append(col)  # Keep non-date columns

        df = df[columns_to_keep]

        # Save DataFrame to CSV
        output_file = os.path.join(folder_name, f"{table_title}.csv")
        df.to_csv(output_file, index=False)
        print(f"Saved {output_file}")

# Process each company in the CSV
for index, row in data.iterrows():
    nse_ticker = row['NSE Ticker']
    listing_date = row['Listing Date']
    try:
        scrape_tables(nse_ticker, listing_date)
    except Exception as e:
        print(f"Failed to scrape {nse_ticker}: {e}")


Table title not found for SBICARD
Table title not found for SBICARD
Table not found for SBICARD in section peer_comparison
Saved SBICARD/quarterly_results.csv
Saved SBICARD/profit_&_loss.csv
Saved SBICARD/balance_sheet.csv
Saved SBICARD/cash_flows.csv
Saved SBICARD/ratios.csv
Table not found for SBICARD in section documents
Table title not found for ROSSARI
Table title not found for ROSSARI
Table not found for ROSSARI in section peer_comparison
Saved ROSSARI/quarterly_results.csv
Saved ROSSARI/profit_&_loss.csv
Saved ROSSARI/balance_sheet.csv
Saved ROSSARI/cash_flows.csv
Saved ROSSARI/ratios.csv
Table not found for ROSSARI in section documents
Table title not found for YESBANK
Table title not found for YESBANK
Table not found for YESBANK in section peer_comparison
Saved YESBANK/quarterly_results.csv
Saved YESBANK/profit_&_loss.csv
Saved YESBANK/balance_sheet.csv
Saved YESBANK/cash_flows.csv
Saved YESBANK/ratios.csv
Table not found for YESBANK in section documents
Table title not found 

In [27]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime

# Load CSV
csv_file = '/Users/hemantg/Desktop/Finance Sem5/mainboard_ipo_final.csv'  # Replace with your CSV file name
data = pd.read_csv(csv_file)

# Define function to scrape tables from Screener

def scrape_tables(nse_ticker, listing_date):
    url = f"https://www.screener.in/company/{nse_ticker}/consolidated/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Convert listing date to datetime
    listing_date = datetime.strptime(listing_date, "%d-%b-%y")

    # Create base folder path
    base_folder = '/Users/hemantg/Downloads/cos. final'
    folder_name = os.path.join(base_folder, nse_ticker)
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Find all tables on the page
    sections = soup.find_all('section', class_='card card-large')

    for section in sections:
        table_title_tag = section.find('h2')
        if not table_title_tag:
            print(f"Table title not found for {nse_ticker}")
            continue

        table_title = table_title_tag.text.strip().replace(' ', '_').lower()
        if 'shareholding_pattern' in table_title:
            continue

        table = section.find('table', class_='data-table')
        if not table:
            print(f"Table not found for {nse_ticker} in section {table_title}")
            continue

        # Extract headers and rows
        headers = [th.text.strip().replace('+', '') for th in table.find('thead').find_all('th')]
        rows = table.find('tbody').find_all('tr')

        data = []
        for row in rows:
            cols = [td.text.strip() for td in row.find_all('td')]
            data.append(cols)

        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)

        # Remove 'Raw PDF' row in Quarterly Results
        if 'quarterly_results' in table_title:
            df = df[df[headers[0]] != 'Raw PDF']
        
        # Filter columns based on Listing Date
        columns_to_keep = [headers[0]]
        for col in headers[1:]:
            try:
                col_date = datetime.strptime(col, "%b %Y")
                if col_date <= listing_date:
                    columns_to_keep.append(col)
            except ValueError:
                columns_to_keep.append(col)  # Keep non-date columns

        df = df[columns_to_keep]

        # Save DataFrame to CSV
        output_file = os.path.join(folder_name, f"{table_title}.csv")
        df.to_csv(output_file, index=False)
        print(f"Saved {output_file}")

# Process each company in the CSV
for index, row in data.iterrows():
    nse_ticker = row['NSE Ticker']
    listing_date = row['Listing Date']
    try:
        scrape_tables(nse_ticker, listing_date)
    except Exception as e:
        print(f"Failed to scrape {nse_ticker}: {e}")


Table title not found for SBICARD
Table title not found for SBICARD
Table not found for SBICARD in section peer_comparison
Saved /Users/hemantg/Downloads/cos. final/SBICARD/quarterly_results.csv
Saved /Users/hemantg/Downloads/cos. final/SBICARD/profit_&_loss.csv
Saved /Users/hemantg/Downloads/cos. final/SBICARD/balance_sheet.csv
Saved /Users/hemantg/Downloads/cos. final/SBICARD/cash_flows.csv
Saved /Users/hemantg/Downloads/cos. final/SBICARD/ratios.csv
Table not found for SBICARD in section documents
Table title not found for ROSSARI
Table title not found for ROSSARI
Table not found for ROSSARI in section peer_comparison
Saved /Users/hemantg/Downloads/cos. final/ROSSARI/quarterly_results.csv
Saved /Users/hemantg/Downloads/cos. final/ROSSARI/profit_&_loss.csv
Saved /Users/hemantg/Downloads/cos. final/ROSSARI/balance_sheet.csv
Saved /Users/hemantg/Downloads/cos. final/ROSSARI/cash_flows.csv
Saved /Users/hemantg/Downloads/cos. final/ROSSARI/ratios.csv
Table not found for ROSSARI in sectio

In [26]:
import os
print(os.access('/Users/hemantg/Desktop/Finance Sem5/cos. final', os.W_OK))


True


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime

# Load CSV
csv_file = '/Users/hemantg/Desktop/Finance Sem5/screener-scraping-try.csv'  # Replace with your CSV file name
data = pd.read_csv(csv_file)

# Define function to scrape tables from Screener

def scrape_tables(nse_ticker, listing_date):
    url = f"https://www.screener.in/company/{nse_ticker}/consolidated/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Convert listing date to datetime
    listing_date = datetime.strptime(listing_date, "%d-%b-%y")

    # Find all tables on the page
    sections = soup.find_all('section', class_='card card-large')

    for section in sections:
        table_title_tag = section.find('h2')
        if not table_title_tag:
            print(f"Table title not found for {nse_ticker}")
            continue

        table_title = table_title_tag.text.strip().replace(' ', '_').lower()
        table = section.find('table', class_='data-table')
        if not table:
            print(f"Table not found for {nse_ticker} in section {table_title}")
            continue

        # Extract headers and rows
        headers = [th.text.strip() for th in table.find('thead').find_all('th')]
        rows = table.find('tbody').find_all('tr')

        data = []
        for row in rows:
            cols = [td.text.strip() for td in row.find_all('td')]
            data.append(cols)

        # Create DataFrame
        df = pd.DataFrame(data, columns=headers)
        
        # Filter columns based on Listing Date
        columns_to_keep = [headers[0]]
        for col in headers[1:]:
            try:
                col_date = datetime.strptime(col, "%b %Y")
                if col_date <= listing_date:
                    columns_to_keep.append(col)
            except ValueError:
                columns_to_keep.append(col)  # Keep non-date columns

        df = df[columns_to_keep]

        # Update column names and row names
        df.columns = [f'{row}_{col}' if col != headers[0] else col for col in df.columns]
        df.insert(0, 'NSE Ticker', nse_ticker)
        df = df.set_index('NSE Ticker')

        # Save DataFrame to CSV
        output_file = f"{nse_ticker}_{table_title}.csv"
        df.to_csv(output_file, index=True)
        print(f"Saved {output_file}")

# Process each company in the CSV
for index, row in data.iterrows():
    nse_ticker = row['NSE Ticker']
    listing_date = row['Listing Date']
    try:
        scrape_tables(nse_ticker, listing_date)
    except Exception as e:
        print(f"Failed to scrape {nse_ticker}: {e}")


In [60]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

def scrape_tables(nse_ticker, listing_date):
    url = f"https://www.screener.in/company/{nse_ticker}/consolidated/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Convert listing date to datetime
    try:
        listing_date = datetime.strptime(listing_date, "%d-%b-%y")
    except ValueError:
        print("\n❌ INCORRECT DATE FORMAT!")
        print("Please use the format: DD-Mon-YY")
        print("Example: 15-Dec-23")
        return

    # Find all tables on the page
    sections = soup.find_all('section', class_='card card-large')

    for section in sections:
        table_title_tag = section.find('h2')
        if not table_title_tag:
            print(f"Table title not found for {nse_ticker}")
            continue

        table_title = table_title_tag.text.strip().replace(' ', '_').lower()
        
        # Skip Shareholding Pattern CSV
        if 'shareholding' in table_title:
            continue

        table = section.find('table', class_='data-table')
        if not table:
            print(f"Table not found for {nse_ticker} in section {table_title}")
            continue

        # Extract headers and rows
        headers = [th.text.strip() for th in table.find('thead').find_all('th')]
        rows = table.find('tbody').find_all('tr')

        data = []
        for row in rows:
            cols = [td.text.strip() for td in row.find_all('td')]
            data.append(cols)

        # Create DataFrame with original headers
        df = pd.DataFrame(data, columns=headers)
        
        # Filter columns based on Listing Date
        columns_to_keep = [headers[0]]
        for col in headers[1:]:
            try:
                col_date = datetime.strptime(col, "%b %Y")
                if col_date <= listing_date:
                    columns_to_keep.append(col)
            except ValueError:
                columns_to_keep.append(col)  # Keep non-date columns

        df = df[columns_to_keep]

        # Remove 'Raw PDF' row for quarterly results
        if 'quarterly_results' in table_title:
            df = df[df.iloc[:, 0] != 'Raw PDF']

        # Save DataFrame to CSV
        output_file = f"{nse_ticker}_{table_title}.csv"
        df.to_csv(output_file, index=False)
        print(f"Saved {output_file}")

def main():
    print("🚀 Screener.in Web Scraper")
    print("=" * 30)
    
    # Get NSE Ticker input
    nse_ticker = input("\nEnter the NSE Ticker (e.g., TATAMOTORS): ").strip().upper()
    
    # Get Listing Date input with format guidance
    listing_date = input("\nEnter Listing Date (Format: DD-Mon-YY, e.g., 15-Dec-23): ").strip()
    
    # Print details entered
    print(f"\n📋 Details Entered:")
    print(f"NSE Ticker: {nse_ticker}")
    print(f"Listing Date: {listing_date}")
    
    # Directly run scraping
    try:
        scrape_tables(nse_ticker, listing_date)
    except Exception as e:
        print(f"Error scraping data: {e}")

if __name__ == "__main__":
    main()

🚀 Screener.in Web Scraper

📋 Details Entered:
NSE Ticker: SAPPHIRE
Listing Date: 18-Nov-21
Table title not found for SAPPHIRE
Table title not found for SAPPHIRE
Table not found for SAPPHIRE in section peer_comparison
Saved SAPPHIRE_quarterly_results.csv
Saved SAPPHIRE_profit_&_loss.csv
Saved SAPPHIRE_balance_sheet.csv
Saved SAPPHIRE_cash_flows.csv
Saved SAPPHIRE_ratios.csv
Table not found for SAPPHIRE in section documents


In [63]:
import yfinance as yf
import pandas as pd

# Function to get quarterly financial results for a stock
def get_quarterly_results(ticker_symbol):
    # Fetch stock data from Yahoo Finance
    ticker = yf.Ticker(ticker_symbol)

    # Get quarterly financial statements
    income_statement = ticker.quarterly_financials
    balance_sheet = ticker.quarterly_balance_sheet
    cash_flow = ticker.quarterly_cashflow

    # Display the results
    print("Income Statement:\n", income_statement)
    print("\nBalance Sheet:\n", balance_sheet)
    print("\nCash Flow Statement:\n", cash_flow)

    # Save to Excel file
    with pd.ExcelWriter(f'{ticker_symbol}_quarterly_results.xlsx') as writer:
        income_statement.to_excel(writer, sheet_name='Income Statement')
        balance_sheet.to_excel(writer, sheet_name='Balance Sheet')
        cash_flow.to_excel(writer, sheet_name='Cash Flow')

    print(f"Data saved to {ticker_symbol}_quarterly_results.xlsx")

# Example usage
ticker_symbol = "BARBEQUE"  # Replace with desired stock symbol
get_quarterly_results(ticker_symbol)


Income Statement:
 Empty DataFrame
Columns: []
Index: []

Balance Sheet:
 Empty DataFrame
Columns: []
Index: []

Cash Flow Statement:
 Empty DataFrame
Columns: []
Index: []
Data saved to BARBEQUE_quarterly_results.xlsx


In [62]:
pip install yfinance


DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/beautifulsoup4-4.10.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.2/948.2 kB 13.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.3 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.8-cp311-cp311-macosx_13_0_arm64.whl size=273332 sha256=03fefb720123aacf7068d8e8d5988bb94e8a47fcf4390f8b4da7064064f4ec82
  Stored in directory: /Users/hemantg/Library/Caches/pip/wheels/ff/6c/1

In [17]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

def scrape_tables(nse_ticker, listing_date):
    url = f"https://www.screener.in/company/{nse_ticker}/consolidated/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Convert listing date to datetime
    try:
        listing_date = datetime.strptime(listing_date, "%d-%b-%y")
    except ValueError:
        print("\n❌ INCORRECT DATE FORMAT!")
        print("Please use the format: DD-Mon-YY")
        print("Example: 15-Dec-23")
        return

    # Find all tables on the page
    sections = soup.find_all('section', class_='card card-large')

    for section in sections:
        table_title_tag = section.find('h2')
        if not table_title_tag:
            print(f"Table title not found for {nse_ticker}")
            continue

        table_title = table_title_tag.text.strip().replace(' ', '_').lower()
        
        # Skip Shareholding Pattern CSV
        if 'shareholding' in table_title:
            continue

        table = section.find('table', class_='data-table')
        if not table:
            print(f"Table not found for {nse_ticker} in section {table_title}")
            continue

        # Extract headers and rows
        headers = [th.text.strip() for th in table.find('thead').find_all('th')]
        rows = table.find('tbody').find_all('tr')

        data = []
        for row in rows:
            cols = [td.text.strip() for td in row.find_all('td')]
            data.append(cols)

        # Create DataFrame with original headers
        df = pd.DataFrame(data, columns=headers)
        
        # Filter columns based on Listing Date
        columns_to_keep = [headers[0]]
        for col in headers[1:]:
            try:
                col_date = datetime.strptime(col, "%b %Y")
                if col_date <= listing_date:
                    columns_to_keep.append(col)
            except ValueError:
                columns_to_keep.append(col)  # Keep non-date columns

        df = df[columns_to_keep]

        # Remove 'Raw PDF' row for quarterly results
        if 'quarterly_results' in table_title:
            df = df[df.iloc[:, 0] != 'Raw PDF']

        # Save DataFrame to CSV
        output_file = f"{nse_ticker}_{table_title}.csv"
        df.to_csv(output_file, index=False)
        print(f"Saved {output_file}")

def main():
    print("🚀 Screener.in Web Scraper")
    print("=" * 30)
    
    # Get NSE Ticker input
    nse_ticker = input("\nEnter the NSE Ticker (e.g., TATAMOTORS): ").strip().upper()
    
    # Get Listing Date input with format guidance
    listing_date = input("\nEnter Listing Date (Format: DD-Mon-YY, e.g., 15-Dec-23): ").strip()
    
    # Print details entered
    print(f"\n📋 Details Entered:")
    print(f"NSE Ticker: {nse_ticker}")
    print(f"Listing Date: {listing_date}")
    
    # Directly run scraping
    try:
        scrape_tables(nse_ticker, listing_date)
    except Exception as e:
        print(f"Error scraping data: {e}")

if __name__ == "__main__":
    main()

🚀 Screener.in Web Scraper

📋 Details Entered:
NSE Ticker: PAYTM
Listing Date: 18-Nov-21
Table title not found for PAYTM
Table title not found for PAYTM
Table not found for PAYTM in section peer_comparison
Saved PAYTM_quarterly_results.csv
Saved PAYTM_profit_&_loss.csv
Saved PAYTM_balance_sheet.csv
Saved PAYTM_cash_flows.csv
Saved PAYTM_ratios.csv
Table not found for PAYTM in section documents


In [19]:
import pandas as pd
import os

# Specify file paths here
xlsx_path = '/Users/hemantg/Downloads/Quarterly Data - Moneycontrol.xlsx'
output_folder = '/Users/hemantg/Downloads/yashvi-csvs'

def split_xlsx_to_csv(xlsx_path, output_folder):
    try:
        # Load the Excel file
        excel_file = pd.ExcelFile(xlsx_path)
        
        # Ensure the output folder exists
        os.makedirs(output_folder, exist_ok=True)
        
        # Iterate through each sheet and save as CSV
        for sheet_name in excel_file.sheet_names:
            df = excel_file.parse(sheet_name)
            csv_file_path = os.path.join(output_folder, f"{sheet_name}.csv")
            df.to_csv(csv_file_path, index=False)
            print(f"Saved: {csv_file_path}")

        print("All sheets have been successfully saved as CSV files.")
        
    except Exception as e:
        print(f"An error occurred: {e}")

# Execute the function
split_xlsx_to_csv(xlsx_path, output_folder)


Saved: /Users/hemantg/Downloads/yashvi-csvs/SBI Cards.csv
Saved: /Users/hemantg/Downloads/yashvi-csvs/Rossari.csv
Saved: /Users/hemantg/Downloads/yashvi-csvs/Yes Bank.csv
Saved: /Users/hemantg/Downloads/yashvi-csvs/Happiest Minds.csv
Saved: /Users/hemantg/Downloads/yashvi-csvs/Route Mobile.csv
Saved: /Users/hemantg/Downloads/yashvi-csvs/CAMS.csv
Saved: /Users/hemantg/Downloads/yashvi-csvs/Chemcon.csv
Saved: /Users/hemantg/Downloads/yashvi-csvs/Angel Broking.csv
Saved: /Users/hemantg/Downloads/yashvi-csvs/Mazagon.csv
Saved: /Users/hemantg/Downloads/yashvi-csvs/UTI AMC.csv
Saved: /Users/hemantg/Downloads/yashvi-csvs/Likhitha Infra.csv
Saved: /Users/hemantg/Downloads/yashvi-csvs/Equitas SFB.csv
Saved: /Users/hemantg/Downloads/yashvi-csvs/Gland Pharma.csv
Saved: /Users/hemantg/Downloads/yashvi-csvs/Restaurant Brands Asia.csv
Saved: /Users/hemantg/Downloads/yashvi-csvs/Mrs. Bectors Foods.csv
Saved: /Users/hemantg/Downloads/yashvi-csvs/Antony Waste Ltd.csv
Saved: /Users/hemantg/Downloads/yas

In [20]:
import os
import pandas as pd

# Define the folder path
folder_path = '/Users/hemantg/Downloads/yashvi-csvs'

# Iterate through all CSV files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Remove 'Unnamed: 0'
        if 'Unnamed: 0' in df.columns:
            df.rename(columns={'Unnamed: 0': ''}, inplace=True)

        # Correct the column names
        new_columns = {}
        for col in df.columns:
            if col != '' and isinstance(col, str):
                new_col = col.replace("'", " ").replace("'", "")
                new_columns[col] = new_col

        df.rename(columns=new_columns, inplace=True)
        
        # Save the updated CSV file
        df.to_csv(file_path, index=False)

print("CSV files updated successfully!")


CSV files updated successfully!
